## input
case 1

In [1]:
# message의 '원문'과 '유해 요소'는 임의로 넣은 것임.
messages = [
    {
        "role": "user",
        "content": """
원문: "그 사람은 정말 구제불능이다."
유해 요소: ['구제불능']
Negative Span: "구제불능"
작업 목표:
1. 주어진 '원문'에서 감지된 '유해 요소'를 기반으로, 문맥과 의미를 유지하며 부드럽게 완화된 표현으로 대체하세요.
2. 완화된 새 문장을 생성하세요.
        """
    }
]


case 2

In [2]:
# 다중 처리 (임시로 lock)
# messages = [
#     {
#         "role": "user",
#         "content": """
# 작업 대상 문장:
# 1. "그 사람은 정말 구제불능이다." (유해 요소: ['구제불능'], Negative Span: "구제불능")
# 2. "그 사람은 정말 무식하다." (유해 요소: ['무식'], Negative Span: "무식")
# 작업 목표:
# 1. 각 문장에서 '유해 요소'를 문맥과 의미를 유지하며 완화된 표현으로 대체하세요.
# 2. 각 문장에 대한 완화된 결과를 생성하세요.
#         """
#     }
# ]


case 3.
감지된 유해 표현과 원무을 구조적으로 분리하여 입력하기

In [3]:
messages = [
    {
        "role": "user",
        "content": """
원문: "그 사람은 정말 구제불능이다."
유해 요소: ['구제불능']
작업 목표:
1. '유해 요소'를 문맥과 의미를 유지하며 완화된 표현으로 대체하세요.
2. 완화된 문장을 반환하세요.
        """
    },
    {
        "role": "user",
        "content": """
원문: "그 사람은 정말 무식하다."
유해 요소: ['무식']
작업 목표:
1. '유해 요소'를 문맥과 의미를 유지하며 완화된 표현으로 대체하세요.
2. 완화된 문장을 반환하세요.
        """
    }
]


최종 메시지 템플릿

In [2]:
messages = [
    {
        "role": "user",
        "content": """
원문: "그 사람은 정말 구제불능이다."
유해 요소: ['구제불능']
Negative Span: "구제불능"
작업 목표:
1. '유해 요소'를 문맥과 의미를 유지하며 완화된 표현으로 대체하세요.
2. 완화된 새 문장을 반환하세요.
결과:"""

    }
]


추가로 긴 문장이 들어왔을 때 핸들링하는 방법도 고려하면 좋을듯.

In [1]:
!pip install transformers accelerate #  모델의 GPU/TPU 가속화와 분산 학습을 도와주는 라이브러리

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")

model.config.pad_token_id = model.config.eos_token_id

print("모델 로딩 완료!")

#모델과 토크나이저를 한 번만 로드한 후, 메모리에 유지합니다.
#이를 위해 별도의 셀에 모델 로드 코드를 작성하고, 이후 실험에서는 해당 객체를 재사용합니다.
# tokenizer.pad_token = tokenizer.eos_token

# input_text = """
# 유해 요소: ['구제불능'].
# 기존 문장: 그런 인간은 정말 구제불능이다.
# 수정된 문장:"""


# inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# output = model.generate(
#     inputs['input_ids'],
#     attention_mask=inputs['attention_mask'],
#     max_new_tokens=50,
#     temperature=0.7,
#     top_k=50,
#     top_p=0.9
# )

# print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

모델 로딩 완료!


In [6]:
#  checkpoint save
# 로컬에 저장
# model.save_pretrained("./local_model")
# tokenizer.save_pretrained("./local_model")


In [7]:
# 구글 드라이브 저장을 위한 mount
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# 저장할 경로 설정
save_path = "/content/drive/MyDrive/yaife/model_checkpoint"

# 모델 및 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"{save_path}에 저장됨.")


/content/drive/MyDrive/yaife/model_checkpoint에 저장됨.


In [9]:
# reuse
# 로컬 디렉터리에서 로드
# tokenizer = AutoTokenizer.from_pretrained("./local_model")
# model = AutoModelForCausalLM.from_pretrained("./local_model")


In [18]:

tokenizer.pad_token = tokenizer.eos_token

#test
input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)


In [19]:
input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

initial_output = model.generate(
    tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)['input_ids'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)
print("Initial Result:", initial_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Initial Result: 
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

변경한 이유: '구제불능'은 단순히 인간이 자신의 문제를 해결할 수 없다는 것을 의미한다. 이는 부정적인 표현이기 때문에 '도�


In [20]:
# Self-reflection 단계
reflection_input = f"""
생성된 문장은 다음과 같습니다:
"{initial_result}"
이 문장은 원래 문장의 의미를 보존하며 긍정적으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘.
"""

reflection_output = model.generate(
    tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

final_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
print("Final Result after Reflection:", final_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Final Result after Reflection: 
생성된 문장은 다음과 같습니다:
"
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

변경한 이유: '구제불능'은 단순히 인간이 자신의 문제를 해결할 수 없다는 것을 의미한다. 이는 부정적인 표현이기 때문에 '도�"
이 문장은 원래 문장의 의미를 보존하며 긍정적으로 변경되었나? 
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘.
문장의 의미는 그대로 유지되지만, 더 긍정적인 표현으로 변경되어 있습니다. 예를 들어, '구제불능'이란 단순히 도움을 받지 못한다는 것을 의미하므로,


In [21]:
#loop 적용 (원하는 답을 생성할 때 까지 - 언어모델 기준)
for _ in range(3):
    reflection_input = f"""
    이전 결과: "{initial_result}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """
    reflection_output = model.generate(
        tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)['input_ids'],
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.9
    )
    initial_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
    print("Updated Result:", initial_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Updated Result: 
    이전 결과: "
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

변경한 이유: '구제불능'은 단순히 인간이 자신의 문제를 해결할 수 없다는 것을 의미한다. 이는 부정적인 표현이기 때문에 '도�"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     이 경우, 문맥에 맞지 않거나 의미가 왜곡된 경우가 아니라면
    문장의 의미를 유지하면서, 긍정적인 표현으로 변경해 주세요. 
    예를 들어, '구제불


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Updated Result: 
    이전 결과: "
    이전 결과: "
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

변경한 이유: '구제불능'은 단순히 인간이 자신의 문제를 해결할 수 없다는 것을 의미한다. 이는 부정적인 표현이기 때문에 '도�"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     이 경우, 문맥에 맞지 않거나 의미가 왜곡된 경우가 아니라면
    문장의 의미를 유지하면서, 긍정적인 표현으로 변경해 주세요. 
    예를 들어, '구제불"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     이 경우, 문맥에 맞지 않거나 의미가 왜곡된 경우가 아니라면
    문장의 의미를 유지하면서, 긍정적인 표현으로 변경해 주세요. 
    예를 들어, '구제불
Updated Result: 
    이전 결과: "
    이전 결과: "
    이전 결과: "
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

변경한 이유: '구제불능'은 단순히 인간이 자신의 문제를 해결할 수 없다는 것을 의미한다. 이는 부정적인 표현이기 때문에 '도�"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 
    바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     이 경우, 문맥에 맞지 않거나 의미가 왜곡된 경우가 아니라면
    문장의 의미를 유지하면서, 긍정적인 표현으로 변경해 주세요. 
    예를 들어, '구제불"
    문맥에 맞지

In [22]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    """두 텍스트 간 유사도를 계산"""
    return SequenceMatcher(None, a, b).ratio()

def is_satisfactory(output):
    """모델의 출력이 적합한지 판단"""
    return "긍정적으로 변경되었습니다" in output or "완료" in output

# 초기값 설정
prev_result = None
MAX_ITERATIONS = 5  # 최대 반복 횟수

for i in range(MAX_ITERATIONS):
    reflection_input = f"""
    이전 결과: "{prev_result if prev_result else "초기 입력"}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """

    inputs = tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        no_repeat_ngram_size=3
    )

    new_result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # 적합한 결과인지 확인
    if is_satisfactory(new_result):
        print(f"Iteration {i+1}: Satisfactory result found, stopping loop.")
        print("Final Result:", new_result)
        break

    # 유사도가 높은 경우 중단
    if prev_result and calculate_similarity(prev_result, new_result) > 0.9:
        print(f"Iteration {i+1}: No significant improvement, stopping loop.")
        print("Final Result:", new_result)
        break

    # 결과 업데이트
    prev_result = new_result
    print(f"Iteration {i+1} Result:", new_result)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 1 Result: 이전 결과: "초기 입력"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     - "아니오, 그럼에도 불구하고" -> "아니다, 그대로는 okay"
    - "그럼에도" -> 
    - "(이제는) 초기 입력" -> (이제) 초인


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 2 Result: 이전 결과: "이전 결과: "초기 입력"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     - "아니오, 그럼에도 불구하고" -> "아니다, 그대로는 okay"
    - "그럼에도" -> 
    - "(이제는) 초기 입력" -> (이제) 초인"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     (이 예제는 문맥이 맞지 않는 경우, 수정해주면 좋습니다.) 
    ```python
# 예제 코드
import math

def calculate_area(radius):
    area = math.pi * (radius ** 2)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 3 Result: 이전 결과: "이전 결과: "이전 결과: "초기 입력"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     - "아니오, 그럼에도 불구하고" -> "아니다, 그대로는 okay"
    - "그럼에도" -> 
    - "(이제는) 초기 입력" -> (이제) 초인"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     (이 예제는 문맥이 맞지 않는 경우, 수정해주면 좋습니다.) 
    ```python
# 예제 코드
import math

def calculate_area(radius):
    area = math.pi * (radius ** 2)"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     # return area
    return area

# 예시 사용
radius = 5
result = calculate_area(5)
print(f"The area of the circle with radius {radius} is {result}.")
``` 
이 코드를 수정해주세요


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Iteration 4 Result: 이전 결과: "이전 결과: "이전 결과: "이전 결과: "초기 입력"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     - "아니오, 그럼에도 불구하고" -> "아니다, 그대로는 okay"
    - "그럼에도" -> 
    - "(이제는) 초기 입력" -> (이제) 초인"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     (이 예제는 문맥이 맞지 않는 경우, 수정해주면 좋습니다.) 
    ```python
# 예제 코드
import math

def calculate_area(radius):
    area = math.pi * (radius ** 2)"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     # return area
    return area

# 예시 사용
radius = 5
result = calculate_area(5)
print(f"The area of the circle with radius {radius} is {result}.")
``` 
이 코드를 수정해주세요"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     ```python 
# 예를 들어, 아래 코드는 math.pi를 math.pi로 사용하고 있어야 합니다.
import math 

def calculate_circle_area(radius): 
    area  = math. pi * ( radius **  2 )
Iteration 5 Result: 이전 결과: "이전 결과: "이전 결과: "이전 결과: "이전 결과: "초기 입력"
    문맥에 

# 시나리오
input).
유해 요소: ['구제불능'].
기존 문장: 그런 인간은 정말 구제불능이다.  
수정된 문장:


output).
Iteration 1 Result: 그런 인간은 정말 도움이 필요하다.
Iteration 2 Result: 그런 인간은 개선 가능성이 있는 사람이다.
Iteration 3: Satisfactory result found, stopping loop.  
Final Result: 그런 인간은 개선 가능성이 있는 사람이다.


In [23]:
print(f"Iteration {i+1} Result:", new_result)

Iteration 5 Result: 이전 결과: "이전 결과: "이전 결과: "이전 결과: "이전 결과: "초기 입력"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     - "아니오, 그럼에도 불구하고" -> "아니다, 그대로는 okay"
    - "그럼에도" -> 
    - "(이제는) 초기 입력" -> (이제) 초인"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     (이 예제는 문맥이 맞지 않는 경우, 수정해주면 좋습니다.) 
    ```python
# 예제 코드
import math

def calculate_area(radius):
    area = math.pi * (radius ** 2)"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     # return area
    return area

# 예시 사용
radius = 5
result = calculate_area(5)
print(f"The area of the circle with radius {radius} is {result}.")
``` 
이 코드를 수정해주세요"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
     ```python 
# 예를 들어, 아래 코드는 math.pi를 math.pi로 사용하고 있어야 합니다.
import math 

def calculate_circle_area(radius): 
    area  = math. pi * ( radius **  2 )"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새

최종 결과물 출력이 깔끔하지않아 아래와 같이 수정함.

In [ ]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    """두 텍스트 간 유사도를 계산"""
    return SequenceMatcher(None, a, b).ratio()

def is_satisfactory(output):
    """모델의 출력이 적합한지 판단"""
    return "긍정적으로 변경되었습니다" in output or "완료" in output

# 초기값 설정
prev_result = None
MAX_ITERATIONS = 5  # 최대 반복 횟수
final_result = None  # 최종 결과 저장

for i in range(MAX_ITERATIONS):
    reflection_input = f"""
    이전 결과: "{prev_result if prev_result else "초기 입력"}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """

    inputs = tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=50,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        no_repeat_ngram_size=3
    )

    new_result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # 적합한 결과인지 확인
    if is_satisfactory(new_result):
        final_result = new_result
        print("Satisfactory result found, stopping loop.")
        break

    # 유사도가 높은 경우 중단
    if prev_result and calculate_similarity(prev_result, new_result) > 0.9:
        final_result = new_result
        print("No significant improvement, stopping loop.")
        break

    # 결과 업데이트
    prev_result = new_result

# 최종 결과 출력
if final_result:
    print("Final Adjusted Sentence:", final_result)
else:
    print("No satisfactory result found.")
